In [2]:
import yfinance as yf
import pandas as pd
import datetime as dt
from pandas.tseries.offsets import MonthEnd

In [9]:
df = pd.read_csv('sp100.csv')
tickers =  [i.replace('.','-') for i in df.Symbol.to_list()]
df = yf.download(tickers,start='2005-01-01')
df

[*********************100%***********************]  101 of 101 completed


Adj Close                                                  \
                  AAPL        ABBV         ABT         ACN        ADBE   
Date                                                                     
2005-01-03    0.971844         NaN   14.956534   19.618639   30.838949   
2005-01-04    0.981825         NaN   14.805904   19.157375   30.024111   
2005-01-05    0.990424         NaN   14.636057   19.082981   29.859142   
2005-01-06    0.991192         NaN   14.966144   18.911869   29.364239   
2005-01-07    1.063362         NaN   15.222524   19.797201   29.384233   
...                ...         ...         ...         ...         ...   
2021-07-21  145.399994  116.989998  118.980003  313.769989  612.260010   
2021-07-22  146.800003  117.540001  118.250000  314.260010  623.679993   
2021-07-23  148.559998  118.190002  120.800003  318.649994  625.869995   
2021-07-26  148.990005  117.790001  119.519997  318.980011  620.799988   
2021-07-27  146.770004  117.959999  119.809998  319.890015  618.280029   

                                                                         ...  \
                   AIG        AMGN         AMT         AMZN        AVGO  ...   
Date                                                                     ...   
2005-01-03  873.705200   50.510906   15.225256    44.520000         NaN  ...   
2005-01-04  876.616089   49.709389   15.024044    42.139999         NaN  ...   
2005-01-05  891.171082   48.923599   14.889898    41.770000         NaN  ...   
2005-01-06  892.627014   49.379353   15.024044    41.049999         NaN  ...   
2005-01-07  894.214600   49.481529   15.292330    42.320000         NaN  ...   
...                ...         ...         ...          ...         ...  ...   
2021-07-21   48.290001  244.850006  280.260010  3585.199951  479.019989  ...   
2021-07-22   47.560001  244.740005  281.600006  3638.030029  476.329987  ...   
2021-07-23   47.310001  247.720001  285.260010  3656.639893  483.149994  ...   
2021-07-26   47.709999  245.080002  284.609985  3699.820068  482.239990  ...   
2021-07-27   47.419998  244.679993  286.880005  3626.389893  475.359985  ...   

             Volume                                                    \
                UNH      UNP       UPS      USB           V        VZ   
Date                                                                    
2005-01-03  6252600  7762400   2176500  5374600         NaN   9236061   
2005-01-04  5582000  8108400   3127400  4552700         NaN  11680234   
2005-01-05  4577600  5504800   2130700  5701600         NaN   8201616   
2005-01-06  4837600  5653600   1773600  4203000         NaN   7844458   
2005-01-07  6653200  3998000   2396900  5312700         NaN   8424450   
...             ...      ...       ...      ...         ...       ...   
2021-07-21  1863700  2320000   1770900  5544800  11241400.0  25264100   
2021-07-22  1421900  4194900   1629700  4042900  10293600.0  16231400   
2021-07-23  1611700  3946800   2188900  3570500   6762000.0  14874700   
2021-07-26  2232600  2840900   2669400  3721300   4875500.0  15328200   
2021-07-27  2630200  3199000  13695700  4332400   7034000.0  15224600   

                                                   
                WBA       WFC       WMT       XOM  
Date                                               
2005-01-03  9757800   8168200  16078400  16009200  
2005-01-04  6618200   9241400  10596700  16718000  
2005-01-05  6850200   7694600   8241500  12189100  
2005-01-06  4816400   8887600  14028800  13098700  
2005-01-07  2946400   6573200   9660300  11315300  
...             ...       ...       ...       ...  
2021-07-21  8544400  29654600   6197300  20908700  
2021-07-22  3519000  23309500   4337800  13493800  
2021-07-23  3742600  23944700   5314100  14081500  
2021-07-26  4055900  22945100   6171000  18374900  
2021-07-27  5119100  22117000   5130200  16846000  

[4170 rows x 606 columns]

In [12]:
df['Volume']

,AAPL,ABBV,ABT,ACN,ADBE,AIG,AMGN,AMT,AMZN,AVGO,...,UNH,UNP,UPS,USB,V,VZ,WBA,WFC,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2005-01-03,691992000,NaN,7165902,2425500,5508800,379590,6657100,2153000,10446500,NaN,...,6252600,7762400,2176500,5374600,NaN,9236061,9757800,8168200,16078400,16009200
2005-01-04,1096810400,NaN,7886828,4101800,7515400,379110,7729200,1999800,19418500,NaN,...,5582000,8108400,3127400,4552700,NaN,11680234,6618200,9241400,10596700,16718000
2005-01-05,680433600,NaN,6601292,3302600,3566600,558800,7482600,1671700,8354200,NaN,...,4577600,5504800,2130700,5701600,NaN,8201616,6850200,7694600,8241500,12189100
2005-01-06,705555200,NaN,11950188,3295400,6159600,408765,6623200,1035100,8700900,NaN,...,4837600,5653600,1773600,4203000,NaN,7844458,4816400,8887600,14028800,13098700
2005-01-07,2227450400,NaN,13965402,10569900,8512400,311070,5381400,1088500,9836600,NaN,...,6653200,3998000,2396900,5312700,NaN,8424450,2946400,6573200,9660300,11315300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-07-21,74993500,4133800.0,4285300,1180300,1381000,3970900,1901900,1167000,2319000,1366000.0,...,1863700,2320000,1770900,5544800,11241400.0,25264100,8544400,29654600,6197300,20908700
2021-07-22,77338200,6257000.0,6958000,1274800,1477900,2815300,1379800,1152200,3265400,1258300.0,...,1421900,4194900,1629700,4042900,10293600.0,16231400,3519000,23309500,4337800,13493800
2021-07-23,71361600,4411100.0,4736100,1221000,2030400,3029100,1747300,1144200,2430700,1348900.0,...,1611700,3946800,2188900,3570500,6762000.0,14874700,3742600,23944700,5314100,14081500


In [13]:
df.to_csv('datasets/sp100.csv.gz', index=True)


In [25]:
import feather
import pyarrow.parquet as pq
import pyarrow as pa

%time df.to_parquet('datasets/sp100.parquet')
#fdf.to_feather('datasets/sp100.feather')

CPU times: user 266 ms, sys: 93.8 ms, total: 359 ms
Wall time: 369 ms
